# Case study of Spirulina trimeric PS-I cores

This case study concerns the fluorescence measured from Spirulina trimeric photosystem I (PS-I) cores. The measurement, the global and target analysis are described in (Gobets et al. 2001). The estimation of the equilibria with the help of an equal area penalty is explained in (Snellenburg et al. 2013).

Gobets B, van Stokkum IHM, Rogner M, Kruip J, Schlodder E, Karapetyan NV, Dekker JP, van Grondelle R (2001) Time-resolved fluorescence emission measurements of photosystem I particles of various cyanobacteria: A unified compartmental model. Biophysical Journal 81 (1):407-424.

Snellenburg JJ, Dekker JP, van Grondelle R, van Stokkum IHM (2013) Functional Compartmental Modeling of the Photosystems in the Thylakoid Membrane at 77 K. The Journal of Physical Chemistry B 117 (38):11363-11371.

# Global analysis
Estimation of the lifetimes, assuming input via a Soret compartment with zero SAS in the Qy region.

### Read and inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

data_path = "streakdata.ascii"
plot_data_overview(data_path, nr_of_data_svd_vectors=4);

## Global Analysis

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

global_model_path = "models/global-model.yml"
global_parameters_path = "models/global-parameters.yml"

#### Model file

In [ ]:
display_file(global_model_path, syntax="yaml")

#### Parameters file

In [ ]:
display_file(global_parameters_path, syntax="yaml")

The free parameters are the kinetic parameters and the IRF parameters. The IRF consists of one Gaussian. The IRF parameters are the center (the location of the maximum of the IRF) and the width of the Gaussian.

### Create scheme and optimize it

In [ ]:
from glotaran.project.scheme import Scheme

global_scheme = Scheme(
    model=global_model_path,
    parameters=global_parameters_path,
    data={"dataset1": data_path},
    maximum_number_function_evaluations=11,
)
global_scheme.validate()

In [ ]:
compartments = global_scheme.model.initial_concentration["input1"].compartments

global_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
from glotaran.optimization.optimize import optimize

global_result = optimize(global_scheme)

In [ ]:
global_result

In [ ]:
global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(global_result.data["dataset1"], wavelength_range=(670, 750))
plot_fitted_traces(global_result, wavelengths, linlog=True, linthresh=20);

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    global_result,
    linlog=False,
    figure_only=False,
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
);

# Target analysis
Estimation of the equilibria between the Bulk and Red Chl compartments with an equal area penalty.

## Target analysis

Theoretical background from ``compartmental_models.pdf``

![model_scheme](./target_scheme.jpg)


Concentration vector

$$c(t)= \begin{bmatrix} S(t) & B(t) & R_1(t) & R_2(t) & F(t)\end{bmatrix}^T$$

The concentration function correspond to

| ***Function name***    | ***S(t)***   | ***B(t)***   | ***R_1(t)***   | ***R_2(t)***   | ***F(t)***   |
|------------------------|--------------|--------------|----------------|----------------|--------------|
| **Name in the schema** | Soret        | Bulk         | Red1           | Red2           | Free         |


Differential equation

$$\dfrac{\mathrm{d}}{\mathrm{dt}}c(t) = \mathbf{K}c(t)+j(t)$$

Input function

$$j(t) = i(t) \begin{bmatrix} 1&0&0&0&0\end{bmatrix}^T$$

Transfer maxtrix $\mathbf{K}$

$$
\mathbf{K} = \begin{bmatrix} 
    -(k_{21} + k_{31} + k_{41} + k_{51}) &                          &                   &                  &       \\ 
    k_{21}                               & -(k_T + k_{32} + k_{42}) & k_{23}            & k_{24}           &       \\
    k_{31}                               &  k_{32}                  & -(k_{F} + k_{23}) &                  &       \\
    k_{41}                               &  k_{42}                  &                   & -(k_{F} + k_{24}) &      \\
    k_{51}                               &                          &                   &                  & k_{F} \\
    \end{bmatrix}
$$

- $k_T$: effective rate constantof Chl, T stands for photochemical Trapping of the excitation energy
- $k_F$: natural decay rate of free Chl, F stands for Fluorescence

# Target analysis
Estimation of the equilibria between the Bulk and Red Chl compartments with an equal area penalty.

### Reduced K-matrix

$$
\mathbf{K} = \begin{bmatrix} 
           &        &        &        &       \\ 
    k_{21} & k_T    & k_{23} & k_{24} &       \\
    k_{31} & k_{32} & k_{F}  &        &       \\
    k_{41} & k_{42} &        & k_{F}  &       \\
    k_{51} &        &        &        & k_{F} \\
    \end{bmatrix}
$$

### Reduced K-matrix ignoring input to Red1 and Red2

$$
\mathbf{K} = \begin{bmatrix} 
           &        &        &        &       \\ 
    k_{21} & k_T    & k_{23} & k_{24} &       \\
           & k_{32} & k_{F}  &        &       \\
           & k_{42} &        & k_{F}  &       \\
    k_{51} &        &        &        & k_{F} \\
    \end{bmatrix}
$$

With the rate relations:

- $k_{23} = a_1~k_{32}$
- $k_{24} = a_2~k_{42}$

### equal_area-target-model.yml

In [ ]:
display_file("models/equal_area-target-model.yml", syntax="yaml")

### equal_area-target-parameters.yml

In [ ]:
display_file("models/equal_area-target-parameters.yml", syntax="yaml")

### Create scheme and optimize it

In [ ]:
target_scheme = Scheme(
    "models/equal_area-target-model.yml",
    "models/equal_area-target-parameters.yml",
    {"dataset1": data_path},
    maximum_number_function_evaluations=7,
)
target_scheme.validate()

In [ ]:
compartments = target_scheme.model.initial_concentration["input1"].compartments

target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
target_result = optimize(target_scheme)

In [ ]:
target_result.data["dataset1"]

### Result plots

In [ ]:
figure, axes = plot_overview(
    target_result,
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    # linthresh=20,
    figure_only=False,
)

In [ ]:
target_result.optimized_parameters